# Installing Libraries

In [ ]:
!pip install -q scikit-learn==1.4.1.post1 numpy pandas altair

# Hard Margin SVM
Scikit-learn의 SVM 관련 분류기는 [sklearn.svm](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.svm)에 존재하며, 그중 C-SVM는 [sklearn.svm.SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)에 구현되어 있다.(Support Vector Classification)

Hard Margin SVM을 위한 별도의 구현체는 없으나, C의 값을 무작정 크게 설정하면 Hard Margin SVM과 동일한 효과를 얻을 수 있다.

일단, Iris 데이터 중 일부만을 활용해서 실제 SVM이 어떻게 초평면을 만들어내는 지 확인해보자.


In [ ]:
from sklearn.datasets import load_iris


data = load_iris(as_frame=True)
X, y = data.data, data.target
I = (y == 0) | (y == 1)
X, y = X.loc[I, ['petal length (cm)', 'petal width (cm)']], y[I]

아래처럼 Linear Kernel을 활용하고, C의 값을 크게 줘서 Hard Margin SVM으로 훈련시킨다.

In [ ]:
model = SVC(kernel='linear', C=1e10).fit(X=X, y=y)

Margin을 이루는 초평면은 $\mathbf{w} \cdot \mathbf{x} + b = w_1 x_1 + w_2 x_2 + b = \pm 1$이므로, 이를 적당히 정리하면,
$$x_2 = -\frac{1}{w_2} \bigg(w_1x_1 + b \pm 1\bigg)$$

아래의 함수는 주어진 입출력과 모델에 대해 학습한 초평면을 시각화하는 함수다:

In [ ]:
import altair as alt
import pandas as pd
import numpy as np


def plot_linear_hyperplane(X, y, model):
    x1_name, x2_name = X.columns[0], X.columns[1]
    y_name = y.name
    w, b = model.coef_[0], model.intercept_[0]
    min_x1, max_x1 = X[x1_name].min(), X[x1_name].max()
    x1_h = np.linspace(min_x1, max_x1, 100)
    x2_h0 = - (w[0] * x1_h + b) / w[1]
    x2_h1 = - (w[0] * x1_h + b + 1) / w[1]
    x2_h2 = - (w[0] * x1_h + b - 1) / w[1]

    df_point = pd.concat([X, y], axis=1)
    df_h0 = pd.DataFrame({x1_name: x1_h, x2_name: x2_h0})
    df_h1 = pd.DataFrame({x1_name: x1_h, x2_name: x2_h1})
    df_h2 = pd.DataFrame({x1_name: x1_h, x2_name: x2_h2})

    p_point = alt.Chart(df_point).mark_point(filled=True).encode(
        x=f'{x1_name}:Q',
        y=f'{x2_name}:Q',
        color=f'{y_name}:N'
    )

    p_h0 = alt.Chart(df_h0).mark_line(color='grey').encode(
        x=f'{x1_name}:Q',
        y=f'{x2_name}:Q',
    )
    p_h1 = alt.Chart(df_h1).mark_line(color='grey', strokeDash=(8, 2)).encode(
        x=f'{x1_name}:Q',
        y=f'{x2_name}:Q',
    )
    p_h2 = alt.Chart(df_h2).mark_line(color='grey', strokeDash=(8, 2)).encode(
        x=f'{x1_name}:Q',
        y=f'{x2_name}:Q',
    )

    return (p_h0 + p_h1 + p_h2 + p_point)

In [ ]:
plot_linear_hyperplane(X, y, model).interactive()

alt.LayerChart(...)

위 그림처럼, 서로 다른 레이블을 구분하는 초평면을 학습했음을 알 수 있다.

# Effect of Feature Scales
SVM은 입력 데이터의 스케일에 민감하다. 스케일이 서로 다르다면, Margin을 최대화할 수 있는 초평면을 학습하기 어렵다. 스케일의 효과를 아래의 극단적인 경우로 확인해보자.

In [ ]:
import altair as alt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler


X = pd.DataFrame({'x1': [1, 5, 3, 5], 'x2': [50, 20, 80, 60]})
y = pd.Series([0, 0, 1, 1], name='y')
model = SVC(kernel='linear', C=1e10).fit(X, y)

X_s = pd.DataFrame(StandardScaler().fit_transform(X), columns=['x1', 'x2'])
model_s = SVC(kernel='linear', C=1e10).fit(X_s, y)


p_unscale = plot_linear_hyperplane(X, y, model).properties(title='Unscaled').interactive()
p_scale = plot_linear_hyperplane(X_s, y, model_s).properties(title='Scaled').interactive()

p_unscale | p_scale

alt.HConcatChart(...)

위에서 볼 수 있듯이, 스케일이 통일되지 않은 SVM은 Margin이 좁게 형성된 것을 알 수 있다. SVM의 목적이 Margin을 최대화하는 것이니, 원하는 대로 학습이 되지 않았다고 할 수 있다.

# Soft Margin SVM
이번엔 Iris 데이터에 임의의 이상치를 집어넣어보자.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.svm import SVC


data = load_iris(as_frame=True)
X, y = data.data, data.target
I = (y == 0) | (y == 1)
X, y = X.loc[I, ['petal length (cm)', 'petal width (cm)']], y[I]

# Outlier 설정
X_outlier, y_outlier = pd.DataFrame({'petal length (cm)': [3.4, 3.2], 'petal width (cm)': [1.3, 0.7]}), pd.Series([0, 0], name='target')
X, y = pd.concat([X, X_outlier], axis=0, ignore_index=True), pd.concat([y, y_outlier], ignore_index=True)

model_hard = SVC(kernel='linear', C=1e2).fit(X, y)
model_med = SVC(kernel='linear', C=1).fit(X, y)
model_soft = SVC(kernel='linear', C=0.1).fit(X, y)

p_hard = plot_linear_hyperplane(X, y, model_hard).properties(title='C=100').interactive()
p_med = plot_linear_hyperplane(X, y, model_med).properties(title='C=1').interactive()
p_soft = plot_linear_hyperplane(X, y, model_soft).properties(title='C=0.1').interactive()

p_hard | p_med | p_soft

alt.HConcatChart(...)

C값이 감소할수록 Margin이 증가하고, 그에 따라 Margin 내 공간에 들어오는 데이터가 많아짐을 알 수 있다.
정확하게 Margin을 경계로 분류가 되지 않기 때문에 훈련 데이터에 대한 학습 오류가 많다는 것을 의미하고, 따라서 과소 적합이 된다는 것을 알 수 있다.

# Kernel Trick
비선형 데이터를 구분하기 위해 Kernel Trick을 적용해보자.

In [ ]:
from sklearn.datasets import make_moons
import pandas as pd
import altair as alt


X, y = make_moons(n_samples=100, noise=0.15, random_state=42)
X, y = pd.DataFrame(X, columns=['x1', 'x2']), pd.Series(y, name='y')
data = pd.concat([X, y], axis=1)

alt.Chart(data).mark_point(filled=True).encode(
    x='x1:Q', y='x2:Q', color='y:N'
)


alt.Chart(...)

## Linear Kernel
$K(\mathbf{x}_i, \mathbf{x}_j) = \mathbf{x}_i \cdot \mathbf{x}_j$

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import log_loss


model = SVC(C=1.0, probability=True, kernel='linear').fit(X, y)
perf = log_loss(y_true=y, y_pred=model.predict_proba(X))
print(f'Log loss = {perf:.5f}')


Log loss = 0.29533


## Polynomial Kernel
$K(\mathbf{x}_i, \mathbf{x}_j) = (\gamma \mathbf{x}_i \cdot \mathbf{x}_j + r)^d$



In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import log_loss


model = SVC(
    C=1.0, probability=True,
    kernel='poly', coef0=1, degree=3, gamma=0.2
).fit(X, y)
perf = log_loss(y_true=y, y_pred=model.predict_proba(X))
print(f'r=1, d=3, gamma=0.2 / Log loss = {perf:.5f}')

model = SVC(
    C=1.0, probability=True,
    kernel='poly', coef0=100, degree=10, gamma='scale'
).fit(X, y)
perf = log_loss(y_true=y, y_pred=model.predict_proba(X))
print(f'r=100, d=10, gamma="scale" / Log loss = {perf:.5f}')

r=1, d=3, gamma=0.2 / Log loss = 0.26910
r=100, d=10, gamma="scale" / Log loss = 0.09939


$\gamma$는 **gamma**, $r$은 **coef0**, $d$는 **degree** 초매개변수로 조정할 수 있다. 확실히, 비선형 데이터에 대해서 Linear Kernel보다 Polynomial Kernel이  성능을 보이는 것을 알 수 있다.

## RBF Kernel
$K(\mathbf{x}_i, \mathbf{x}_j) = \exp(-\gamma ||\mathbf{x}_i - \mathbf{x}_j||^2 )$

RBF Kernel은 SVC 구현체의 기본값으로 설정되어 있을 정도로 가장 흔히 쓰는 Kernel 중 하나다. RBF Kernel의 장점 중 하나는 추가적인 초매개변수가 하나만 존재한다는 것이다.


In [ ]:
model = SVC(
    C=1.0, probability=True,
    kernel='rbf', gamma=0.2
).fit(X, y)
perf = log_loss(y_true=y, y_pred=model.predict_proba(X))
print(f'gamma=0.2 / Log loss = {perf:.5f}')

model = SVC(
    C=1.0, probability=True,
    kernel='rbf', gamma='scale'
).fit(X, y)
perf = log_loss(y_true=y, y_pred=model.predict_proba(X))
print(f'gamma="scale" / Log loss = {perf:.5f}')

gamma=0.2 / Log loss = 0.27528
gamma="scale" / Log loss = 0.10637


## Sigmoid Kernel
$K(\mathbf{x}_i, \mathbf{x}_j) =\tanh(-\gamma \mathbf{x}_i \cdot \mathbf{x}_j + r )$


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import log_loss


model = SVC(
    C=1.0, probability=True,
    kernel='sigmoid', coef0=1, gamma=0.2
).fit(X, y)
perf = log_loss(y_true=y, y_pred=model.predict_proba(X))
print(f'r=1, gamma=0.2 / Log loss = {perf:.5f}')

model = SVC(
    C=1.0, probability=True,
    kernel='sigmoid', coef0=10, gamma='scale'
).fit(X, y)
perf = log_loss(y_true=y, y_pred=model.predict_proba(X))
print(f'r=10, gamma="scale" / Log loss = {perf:.5f}')

r=1, gamma=0.2 / Log loss = 0.31346
r=10, gamma="scale" / Log loss = 0.69480


# SVM의 종류
지금까지 한 것은 C-SVM의 구현체인 [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) 였지만, 실제 [sklearn.svm](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.svm)내에는 다양한 SVM 변형들이 존재한다.

* [sklearn.svm.LinearSVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC), [sklearn.svm.LinearSVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVR.html#sklearn.svm.LinearSVR): Linear Kernel에 Regularization을 활용할 수 있는 SVM 기반 분류 및 회귀 모델
* [sklearn.svm.SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC), [sklearn.svm.SVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR): C-SVM 기반 분류 및 회귀 모델
* [sklearn.svm.NuSVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.NuSVC.html#sklearn.svm.NuSVC), [sklearn.svm.NuSVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.NuSVR.html#sklearn.svm.NuSVR): $\nu$-SVM 기반 분류 및 회귀 모델
